# Module 5 - Deploying A Machine Learning Model

In the previous 2 modules we have worked building and evaluating a model for churn. Now we will collate all of that code and prepare it for deployment. This includes:
- Training the model with KFold CV
- Saving the model
- Loading the model for new predictions on unseen data

In [24]:
# Imports
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

## Data Preparation

In [25]:
df = pd.read_csv("./../data/raw/Customer-Churn-Data.csv")
df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')
df.totalcharges = df.totalcharges.fillna(0)

df.churn = (df.churn == 'yes').astype(int)

In [26]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [27]:
numerical =  ['tenure', 'monthlycharges', 'totalcharges']

categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [28]:
def train(df_train, y_train, C=1.0):
    dicts = df_train[categorical + numerical].to_dict(orient='records')
    
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts)

    model = LogisticRegression(C=C, max_iter=10000)
    model.fit(X_train, y_train)

    return dv, model

In [29]:
def predict(df, dv, model):
    dicts = df[categorical + numerical].to_dict(orient='records')
    X = dv.transform(dicts)
    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [30]:
C = 1.0
n_splits = 5

In [31]:
scores = []

kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

for train_idx, val_idx in kfold.split(df_full_train):

    df_train = df_full_train.iloc[train_idx]
    df_val = df_full_train.iloc[val_idx]

    y_train = df_train.churn.values
    y_val = df_val.churn.values
    
    dv, model = train(df_train, y_train, C=C)
    y_pred = predict(df_val, dv, model)

    auc = roc_auc_score(y_val, y_pred)
    scores.append(auc)

print(f'C={C} {np.mean(scores):.3f} +- {np.std(scores):.3f}')

C=1.0 0.842 +- 0.007


In [32]:
scores

[0.8443375120691413,
 0.8449977651966627,
 0.8333012241621514,
 0.8347768913474451,
 0.8518049323836717]

In [33]:
dv, model = train(df_full_train, df_full_train.churn.values, C=1.0)
y_pred = predict(df_test, dv, model)

y_test = df_test.churn.values
auc = roc_auc_score(y_test, y_pred)
auc

0.8584032088573997

## Saving the model

In [39]:
import pickle
from datetime import datetime

In [42]:
timestamp = datetime.now().strftime('%d_%b_%Y_%H_%M')

output_file_name = f'./../models/model_C={C}_{timestamp}.bin'
output_file_name

'./../models/model_C=1.0_25_Oct_2025_22_32.bin'

In [43]:
with open(output_file_name, 'wb') as f:
    pickle.dump((dv, model), f)

## Load the model

In [1]:
import pickle

In [2]:
input_file_path = './../models/model_C=1.0_25_Oct_2025_22_32.bin'

In [5]:
with open(input_file_path, 'rb') as f:
    dv, model = pickle.load(f)

In [6]:
dv, model

(DictVectorizer(sparse=False), LogisticRegression(max_iter=10000))

In [7]:
customer = {
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'yes',
    'dependents': 'no',
    'phoneservice': 'no',
    'multiplelines': 'no_phone_service',
    'internetservice': 'dsl',
    'onlinesecurity': 'no',
    'onlinebackup': 'yes',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'no',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 29.85,
    'totalcharges': 29.85
}

In [9]:
x = dv.transform([customer])

In [10]:
model.predict_proba(x)[0, 1]

np.float64(0.6275953527536646)